# Interlude: First notes on coding style
Source: [Python documentation on python.org](https://docs.python.org/3/tutorial/controlflow.html#id2)

Now that you are about to write longer, more complex pieces of Python, it is a good time to talk about coding style. Most languages can be written (or more concise, formatted) in different styles; some are more readable than others. Making it easy for others to read your code is always a good idea, and adopting a nice coding style helps tremendously for that.

For Python, [PEP 8](https://peps.python.org/pep-0008/) has emerged as the style guide that most projects adhere to; it promotes a very readable and eye-pleasing coding style. Every Python developer should read it at some point; here are the most important points extracted for you:

- Use 4-space indentation, and no tabs.
  - 4 spaces are a good compromise between small indentation (allows greater nesting depth) and large indentation (easier to read). Tabs introduce confusion, and are best left out.
- Wrap lines so that they don’t exceed 79 characters. \[NOTE: debatable, you could also agree on e.g. 120 characters given todays screens\]
  - This helps users with small displays and makes it possible to have several code files side-by-side on larger displays.
- Use blank lines to separate functions and classes, and larger blocks of code inside functions.
  - \[NOTE: Typically: 2 lines between pure functions, one line between methods in a class\]
- When possible, put comments on a line of their own.
- Use docstrings.
- Use spaces around operators and after commas, but not directly inside bracketing constructs: `a = f(1, 2) + g(3, 4)`.
- Name your classes and functions consistently; the convention is to use **UpperCamelCase for classes** and **lowercase_with_underscores for functions and methods**. Always use self as the name for the first method argument (see A First Look at Classes for more on classes and methods).
- Use UTF-8 encoding for any source files.
- Still though, don’t use non-ASCII characters in identifiers if there is only the slightest chance people speaking a different language will read or maintain the code.

<div class="alert alert-block alert-info">
<b>Further Reading:</b> <br>
    Once again: Code is read <b>way</b> more often than written. Anything that makes reading code easier is really really helpful! A consistent style throughout the codebase is one of those things.<br><br>
    If you'd like to make sure your code is following a good style look into automatic linters. My recommendation would be <a href=https://flake8.pycqa.org/en/latest/>flake8</a>.
</div>

# Scoping Revisited
- refers to the rules that determine the visibility and accessibility of variables and objects within a program.
- crucial for managing the lifetime of variables and avoiding naming conflicts.
- hierarchical scoping system, which means that variables can be defined and accessed within various levels of the program
- the scope in which a variable is defined influences where it can be accessed. 

#### Local Scope (or Function Scope): 
- Variables defined within a function are considered local to that function.
- They are only accessible within the function where they are defined.
- Local scope is the most restrictive, and variables declared within a function do not affect variables with the same name in other parts of the code.

In [ ]:
def my_function():
    x = 10  # Local scope variable
    print(x)

my_function()
# print(x)

#### Enclosing Scope (or Non-local Scope): 
- Variables defined in an enclosing function (a containing function) can be accessed by nested functions within it. This is known as "closure."
- Variables defined in the enclosing function are also known as `free` variables from the perspective of the inner function.
- If you want to assign to such a variable in the inner function you need to declare them as `nonlocal`
- Mutable types can still be modified without assignment

In [ ]:
def outer_function():
    y = 20  # Enclosing scope variable for `inner_function`; local scope variable for `outer_function`

    def inner_function():
        print(y)  # Accessing 'y' from the enclosing scope

    inner_function()

outer_function()

#### Global Scope: 
- Variables defined at the top level of a `module` (file) are considered global and can be accessed from anywhere within the module.
- These variables can also be accessed within functions, but if you want to assign to them within a function, you need to declare them as global using the `global` keyword.
- Changes that do not rely on assignment (for mutable objects) also don't need the `global` keyword

In [ ]:
global_variable = 30  # Global scope variable

def my_function():
    print(global_variable)

my_function()

#### Built-in Scope: 
- Python also has a built-in scope that contains pre-defined functions and objects available globally, such as print(), len(), and others.

### Advanced scoping examples

#### Hierarchy

In [ ]:
def my_function():
    print = 5
    print('foo')

try:
    my_function()
except TypeError as e:
    print(f'Whoa, an error occured: {e}')

print('Hey, print still works here')

In [ ]:
def outer_function():
    print = 'foo'
    def inner_function():
        print('hey there!')
    return inner_function

some_function = outer_function()
print('at this point print still works')

try:
    some_function()
except TypeError as e:
    print(f'Whoa, an error occured: {e}')

print('And now it works again...')

In [ ]:
def f1(a):
    print(a, end=' ')
    print(b)

In [ ]:
f1('Hey')

In [ ]:
b = 'there!'
f1('Hey')  # scope is searched at runtime, not during declaration of the function (see also when the errror above occurs)
del b

In [ ]:
b = 'there!'
def f2(a):
    print(a, end=' ')
    print(b)
    b = 'you!'

f2('Hey')

> design choice, python assumes variables assigned to in the body of a function are `local` to that function. Intentional behaviour to avoid accidentally overwriting global variables

In [ ]:
b = 'there!'
def f3(a):
    global b
    print(a, end=' ')
    print(b)
    b = 'you!'

f3('Hey')
print(f'{b=}')

In [ ]:
# the behaviour seems different for mutables, because we're not assigning to the variable itself, just changing its contents
b = ['there!']
def f2_mutable(a):
    # no `global b` here...
    print(a, end=' ')
    print(b[0])
    b[0] = 'you!'

f2_mutable('Hey')
print(f'{b=}')

In [ ]:
# the same logic applies to closures and their free variables
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [ ]:
my_averager = make_averager()
my_averager(10)

In [ ]:
# but the variables defined in the outer function are not globals
def make_averager_global():
    count = 0
    total = 0
    def averager(new_value):
        global count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [ ]:
my_averager_global = make_averager_global()
my_averager_global(10)

In [ ]:
# instead, the keyword here is nonlocal!
def make_averager_nonlocal():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [ ]:
my_averager_global = make_averager_nonlocal()
my_averager_global(10)
my_averager_global(11)
my_averager_global(12)
my_averager_global(13)

In [ ]:
# summary
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

# Classes and Object Oriented Programming

- Object-Oriented Programming (OOP) is a programming paradigm that revolves around the concept of objects.
- In OOP, software is organized as a collection of objects, each of which represents a real-world entity or concept.
- **Classes**: A class is a blueprint or template for creating objects.
  - defines the structure and behavior that objects of that class will have.
  - if you have a class called Car, objects created from this class will have attributes like make, model, and methods like start_engine().
- **Attributes**: Attributes are data members of a class.
  - represent the characteristics or properties of objects created from that class.
  - for a Car class, attributes might include make, model, year, and so on.
- **Methods**: Methods are functions defined within a class.
  - describe the behavior or actions that objects of that class can perform.
  - for a Car class, methods might include start_engine(), accelerate(), and brake().
- **Objects**: Objects are instances of classes, and the central building blocks of OOP.
  - represent tangible or abstract entities
  - encapsulate both data (attributes) and the behavior (methods) associated with those entities.
  - hold actual values for the placeholders defined in the template/class

#### Further concepts
- **Encapsulation**: Encapsulation is the concept of bundling data (attributes) and methods that operate on that data into a single unit (the class). It allows for information hiding, where the internal implementation details are not exposed to the outside world. This enhances security and maintains data integrity.
- **Inheritance**: Inheritance is a mechanism that allows one class (the subclass or derived class) to inherit the attributes and methods of another class (the superclass or base class). It promotes code reusability and the creation of specialized classes that build upon more general ones.
- **Polymorphism**: Polymorphism allows objects of different classes to be treated as objects of a common superclass. This simplifies code and promotes flexibility. In Python, polymorphism often involves method overriding, where a subclass provides its own implementation of a method inherited from a superclass.
- **Abstraction**: Abstraction involves simplifying complex systems by breaking them into smaller, more manageable parts. It hides the implementation details of a class and exposes only essential features to the user. Abstraction is a fundamental principle of OOP that makes code more understandable and maintainable.

#### Advantages
- **Modularity**: Code is divided into classes, making it easier to manage and understand.
- **Reusability**: You can reuse classes in different parts of your code or in other projects.
- **Maintainability**: Changes can be made to one class without affecting the entire codebase.
- **Encapsulation**: Data is protected from unauthorized access or modification.
- **Real-world modeling**: OOP allows you to model real-world entities and relationships directly in your code.

## Basics of Classes
- created by a `class <name>:` header
- class body indented one level
- so the header of any class method is indented by one level already, and the body one further level
- **constructor**: a method called when the class in 'instantiated', i.e. when an object of that class is created
  -   special method with name `__init__`
- all regular methods receive as first argument a reference to the **instance** they are 'called from'. This first parameter is (by convention) called `self`

### Class construction, attributes and methods

#### class construction

In [ ]:
class Car:
    # constructor, called automatically when instantiating the class
    def __init__(self, make, model):
        # attributes
        # all attributes should (by convention) be initialized in `__init__`, even if not required for the functionality
        self.make = make
        self.model = model

    # method
    def describe(self):  # this is a method, it receives the instance as first attribute
        print(f"I'm a shiny new {self.make} {self.model}")

# creating instances
my_car = Car('Toyota', 'Camry')
your_car = Car('Ford', 'Focus')
print(my_car)

In [ ]:
# creating another instance
new_car = Car('Toyota', 'Camry')
new_car == my_car

In [ ]:
# calling a method; note we're not passing the first argument explicitly
my_car.describe()

In [ ]:
my_car.describe  # instance attribute `describe` is a `bound method`, not a function!

In [ ]:
Car.describe  # class attribute `describe` otoh is a `function`

In [ ]:
some_fun = my_car.describe  # after assignment it's still a method...
some_fun

In [ ]:
some_fun()  # still without having to pass `self`

In [ ]:
# I could (but usually should not) also fill in the reference to the instance myseld instead of relying on the binding mechanism
Car.describe(my_car)

In [ ]:
class MobilePhone:
    def __init__(self):
        self.make = 'Google'
        self.model = 'Pixel 8'

Car.describe(MobilePhone())  # I /could/ call it with anything -- a case of duck typing

In [ ]:
# you can call methods from other methods, of course
class Bag:
    def __init__(self):
        self.data = []

    def add(self, x):
        self.data.append(x)

    def addtwice(self, x):
        self.add(x)
        self.add(x)

bag = Bag()
bag.addtwice('foo')
print(bag.data)

In [ ]:
# weird detail: you can define the /method/ outside the class definition, too...
# Function defined outside the class. Don't though...
def f1(self):
    print(f'f1 was called with the argument: {self}')

class C:
    pass

C.f2 = f1

c = C()
print(f'c.f2 is a {c.f2}')

c.f2()  # no need for `self` here!
print(c)

In [ ]:
# but it doesn't work if assigned to the instance -- binding only happens for functions that are class attributes
c.f3 = f1
print(f'c.f3 is a {c.f3}')
c.f3()

In [ ]:
# instance methods are objects with attributes, too
print(c.f2.__self__, c.f2.__func__)

#### 'protected' or 'private' member attributes
- don't really exist in python (but don't /really really/ exist in e.g. java either...)
- class attributes with leading `_` are considered 'private' by convention only
- class attributes with leading `__` are protected by name mangling

In [ ]:
# encapsulation: protected and private class members
class DemoClass:
    def __init__(self, v1, v2):
        self._protected = v1
        self.__private = v2

    def print_values(self):
        print(f'{self._protected=}, {self.__private=}')

demo_instance = DemoClass(3, 5)
demo_instance.print_values()

In [ ]:
# underscore-attributes are internal implementation detail
# of the class and should not be accessed externally
# convention only, no enforcement
print(demo_instance._protected)
demo_instance._protected = 7
print(demo_instance._protected)

In [ ]:
# double-underscore-attributes are 'private' and can not be seen from the outside;
# achieved only through name-mangling though. Used primarily in inheritance

In [ ]:
print(demo_instance.__private)

In [ ]:
print(demo_instance._DemoClass__private)

In [ ]:
demo_instance._DemoClass__private = 9
demo_instance.print_values()

#### Properties > Attributes

In [ ]:
# better: property decorator
class PropertyDemo:
    def __init__(self):
        self._secret_value = 42

    @property
    def secret_value(self):
        print('Are you really trying to read my secret value?')
        return self._secret_value // 2

    @secret_value.setter
    def secret_value(self, value):
        raise AttributeError("You are most definitely not allowed to change my secret value!")
        

property_demo_instance = PropertyDemo()

read_value = property_demo_instance.secret_value
print(f'I think the secret value is {read_value}')

In [ ]:
property_demo_instance.secret_value = '33'

In [ ]:
# of course you /can/ still access the 'real' attribute though
# this is not a security measure, it just protects from incidental misuse of your code
stolen_value = property_demo_instance._secret_value
print(f'The secret value really is {stolen_value}')

In [ ]:
# and here's a fun example of using deleters on properties

class TheBlackKnight:
    def __init__(self):
        self.members = ["an arm", "another arm", "a leg", "another leg"]
        self.phrases = ["'Tis but a scratch.", "It's just a flesh wound.", "I'm invincible!", "All right, we'll call it a draw"]

    @property
    def member(self):
        print(f'next member is: {self.members[0]}')

    @member.deleter
    def member(self):
        print(f'BLACK KNIGHT (loses {self.members.pop(0)})\n-- {self.phrases.pop(0)}')

In [ ]:
knight = TheBlackKnight()
knight.member

In [ ]:
del knight.member

In [ ]:
del knight.member

In [ ]:
del knight.member

In [ ]:
del knight.member

<div class="alert alert-block alert-info">
<b>Tip:</b> You can change a member attribute from direct access to access through getters and setters without changing the external API!
</div>

#### Class and instance variables
- instance variables are for attributes and methods shared by all instances
- instance attributes are for attributes specific to the instance

In [ ]:
class Dog:
    kind = 'canine'
    
    def __init__(self, name):
        self.name = name

    def describe(self):
        print(f"My name is {self.name} and I'm a {self.kind}")  # access to class variables through instance

my_dog = Dog('Senta')
your_dog = Dog('Fido')

my_dog.describe()
your_dog.describe()

print('\nAfter setting the class attribute to "fish":')

Dog.kind = 'fish'
my_dog.describe()
your_dog.describe()

In [ ]:
print('\nAfter setting the instance attribute back to "canine":')

my_dog.kind = 'canine'
my_dog.describe()
your_dog.describe()
print(f'{Dog.kind=}, {my_dog.kind=}, {your_dog.kind=}')

## Advanced Topics

### Inheritance
- allows you to create a new class based on an existing class.
- The new class inherits the attributes and methods of the existing class, which is often referred to as the "base class" or "superclass."
- Any attribute/method of the base class can be overridden in a derived class to adapt the behaviour to the specifics of the subclass
- The new class is called the "derived class" or "subclass."
- In Python, inheritance enables you to create a hierarchy of classes, where more specific classes inherit properties and behaviors from more general classes.



- **Base Class (Superclass)**: The base class is the class whose attributes and methods are inherited. It is defined as usual with its attributes and methods.
- **Derived Class (Subclass)**: The derived class is the new class that inherits from the base class. It is created by specifying the base class as an argument in the class definition.
- **Syntax**: To create a subclass that inherits from a superclass, you define the subclass with the superclass in parentheses after the class name.<br>
  `class DerivedClass(BaseClass):`
- **Method Overriding**: The derived class can override (replace) methods of the base class with its own implementations. This allows the derived class to customize the behavior of inherited methods while retaining the same method names.
- **Access to Superclass Methods**: A derived class can access the methods and attributes of the base class using the super() function, which provides a way to call methods of the base class from the derived class.
- **Multiple Inheritance**: Python supports multiple inheritance, where a class can inherit from multiple base classes. This allows for greater flexibility but requires careful design to avoid ambiguity. In the case of multiple inheritance, Python follows a specific order to resolve method calls. The super() function is used in this context to determine which superclass's method should be called.

In [ ]:
class Animal:
    def __init__(self, name):
        self.name = name

    def speak(self):
        print(f"{self.name} doesn't know what to say...")

class Dog(Animal):
    def speak(self):
        print(f"{self.name} says Woof!")

class Cat(Animal):
    def speak(self):
        print(f"{self.name} says Meow!")

some_animal = Animal("Nemo")
my_dog = Dog("Buddy")
my_cat = Cat("Whiskers")

some_animal.speak()
my_dog.speak()
my_cat.speak()

In [ ]:
class GoldenRetriever(Dog):
    def describe(self):
        print("Golden Retrievers are friendly and intelligent dogs known for their beautiful golden-colored coats.")

goldie = GoldenRetriever("goldie")  # constructor from base class (`Animal`)
goldie.speak()  # from first derived class (`Dog`)
goldie.describe()  # from second derived class (`GoldenRetriever`)


In [ ]:
# multiple inheritance

class Mammal:
    def mammal_info(self):
        print('Mammals can give direct birth.')

    def generic_info(self):
        print("I'm a mammal")

class WingedAnimal:
    def winged_animal_info(self):
        print('Winged animals can flap.')

    def generic_info(self):
        print("I'm a winged animal")

class Bat(Mammal, WingedAnimal):
    pass

# create an object of Bat class
bat = Bat()

bat.mammal_info()
bat.winged_animal_info()

In [ ]:
# method resolution order determines which parents method gets called
bat.generic_info()

In [ ]:
# multiple inheritance

class Animal:
    def generic_info(self):
        print("I'm just an animal")

class Mammal(Animal):
    def generic_info(self):
        super().generic_info()
        print("I'm a mammal")

class WingedAnimal(Animal):
    def generic_info(self):
        super().generic_info()
        print("I'm a winged animal")

class Bat(Mammal, WingedAnimal):
    def generic_info(self):
        super().generic_info()
        print("I'm a bat")

# create an object of Bat class
bat = Bat()

bat.generic_info()  # just an animal is printed only once

In [ ]:
print(Bat.mro())

In [ ]:
# what do you expect for this?
Mammal.generic_info(bat)

### static- and classmethods

In [ ]:
class Demo:
    def normal_method(*args):
        print(args)

    @classmethod
    def class_method(*args):
        print(args)

    @staticmethod
    def static_method(*args):
        print(args)

In [ ]:
print(Demo.static_method)
Demo.static_method()

In [ ]:
print(Demo.class_method)
Demo.class_method()

In [ ]:
print(Demo.normal_method)
Demo.normal_method()

In [ ]:
demo = Demo()
print(demo.normal_method)
demo.normal_method()

In [ ]:
bytes([5, 7])

### dunder Methods

In [ ]:
class SimpleVector2D:
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

In [ ]:
import math

class Vector2D(SimpleVector2D):
    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __str__(self):
        return str(tuple(self))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            length = abs(self)
            angle = math.atan2(self.y, self.x)
            coords = (length, angle)
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    def __neg__(self):
        return Vector2D(-x for x in self)

    def __abs__(self):
        return math.sqrt(self.x*self.x + self.y*self.y)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __lt__(self, other):
        return abs(self) < abs(other)

    def __le__(self, other):
        return abs(self) <= abs(other)

    def __add__(self, other):
        print('Vector __add__ called')
        return Vector2D(self.x + other.x, self.y + other.y)

    def __radd__(self, other):
        print('Vector __radd__ called')
        return self + other

    def __iadd__(self, other):
        print('Vector __iadd__ called')
        raise TypeError('Inplace modification of vectors is not supported')

    def __sub__(self, other):
        return Vector2D(self.x - other.x, self.y - other.y)

    def __mul__(self, other):
        if isinstance(other, Vector2D):
            # we interpret this as inner product
            # maybe better would be the explicity `__matmul__`-infix operator `@`
            return self.x*other.x + self.y*other.y

        return Vector2D(self.x*other, self.y*other)
    
    def __bool__(self):
        return self.x != 0 or self.y != 0

    def __hash__(self):
        print('Vector __hash__ called')
        return hash((self.x, self.y))

In [ ]:
# string conversion and repr
v1_simple = SimpleVector2D(3, 5)
print('SimpleVector2D:')
print(v1_simple)
print(f'{v1_simple!r}')

v1 = Vector2D(3, 5)
print('\nVector2D:')
print(v1)
print(f'{v1!r}')

In [ ]:
# repr should be built such that `eval`uating the string should yield back the object
# eval: not discussed yet, runs the string passed in as python code. dangerous though!
repr_str = repr(v1)
eval(repr_str)

In [ ]:
# adding custom format options!
print(f'{v1}')
print(f'{v1:p}')
print(f'{v1:.3p}')

In [ ]:
# __iter__ allows conversion to built-in sequence types
print(list(v1))
print(tuple(v1))
print(set(v1))

In [ ]:
# of course that doesn't work for the simple vector
list(v1_simple)

In [ ]:
# comparing two vectors
v1 = Vector2D(3, 5)
v2 = Vector2D(3.0, 5.0)
v3 = Vector2D(5, 3)
print(f'{(v1 == v2) = }')
print(f'{(v1 == v3) = }')


In [ ]:
# different objects compare as unequal by default, only identical objects (same `id`) compare as equal
v1_simple = SimpleVector2D(3, 5)
v2_simple = SimpleVector2D(3, 5)
v3_simple = v1_simple
print(f'{(v1_simple == v2_simple) = }')
print(f'{(v1_simple == v3_simple) = }')

In [ ]:
# mixed case:
v1 == v1_simple

In [ ]:
# comparison to another iterable
# whether this is the desired behaviour is up to you...
v1 == (3, 5)

In [ ]:
# also works the other way around -- why!?
(3, 5) == v1

In [ ]:
class TestClass:
    def __eq__(self, other):
        print('Equality comparison called!')
        return True

test_class = TestClass()

test_class == (1, 2, 3)

In [ ]:
# but why does this work?
(1, 2, 3) == test_class
[1, 2, 3] == test_class

In [ ]:
(1, 2, 3).__eq__(test_class)

In [ ]:
# inequality -- not implemented above, but works anyway?!
v1 != v2

In [ ]:
v1 = Vector2D(3, 5)
v2 = Vector2D(-7, 9)
print(f'{(v1 < v2) = }')
print(f'{(v1 <= v2) = }')
print(f'{(v1 > v2) = }')  # but we never implemented these?
print(f'{(v1 >= v2) = }')

In [ ]:
v2 > v1

In [ ]:
# vectors can be added thanks to __add__
v1 + v2

In [ ]:
# but only if the other thing is also a vector
v1 + 5

In [ ]:
# or at least looks like one (duck typing)...
v1 + v1_simple

In [ ]:
# also works the other way around
# same kind of logic as with __eq__ above...
v1_simple + v1

In [ ]:
v1 += v2

In [ ]:
# same for substraction
v1 - v2

In [ ]:
# but we didn't implement `__rsub__`
v1_simple - v1

In [ ]:
# since we didn't explicitly implement `__isub__` either this actually works...
# that's because if `__isub__` is not implemented, python tries to do an explicit `v1 = v1 - v1_simple` here

v1 -= v1_simple
v1

In [ ]:
# multiplication by a scalar yields another vector
v1 * 5

In [ ]:
# but multiplication of two vectors is here interpreted as inner product and yields a scalar
v1 * v2

In [ ]:
v1 = Vector2D(3, 5)
v2 = Vector2D(0, 0)

if v1:
    print('v1 is truthy')
else:
    print('v1 is falsy')

if v2:
    print('v2 is truthy')
else:
    print('v2 is falsy')

In [ ]:
print(f'{bool(v1)=}, {bool(v2)=}')

In [ ]:
v1 = Vector2D(3, 5)
v2 = Vector2D(0, 0)
v3 = Vector2D(3, 5)
v4 = Vector2D(5, 3)

In [ ]:
my_set = set([v1, v2, v3, v4])
my_set

In [ ]:
# of course if you secretly do change the value things break
print(f'{(v2 in my_set)=}')
v2._SimpleVector2D__x = 1
print(f'{(v2 in my_set)=}')

#### Overview: Special methods for classes:
- **instance creation and destruction**
  - `__init__`: instance constructor, shown above
  - `__del__`: instance destructor
  - `__new__`: called before new instance is created, should return the instance; 
- **str/bytes representation**
  - `__repr__`: compute 'official' string representation; should look like a valid Python expression that could be used to recreate an object with the same values.
  - `__str__`: 'informal' or nicely printable string representation of an object. The return value must be a string object.
  - `__bytes__`: compute a byte-string representation of an object. This should return a bytes object.
  - `__format__`: Called by the format() built-in, and by extension, evaluation of f-strings to produce a 'formatted' string representation of an object
- **comparison operators**
  - `__lt__` (<), `__le__` (<=), `__eq__` (==), `__ge__` (>=), `__gt__` (>), `__ne__` (!=): 'rich comparison' operators; should return `True` or `False`;
- **hash calculation**
  - implemented by defining the `__hash__`-method, should return an integer
  - equal hashes /must/ imply objects compare equal (but not vice versa)
  - if `__eq__` is not defined you also should not define `__hash__`
  - recommendation: stick all relevant attributes into a tuple and calculate the hash of the tuple
  - only for 'immutable' objects; if the values can change lookups in dicts/sets can break
- **type conversions**
  - `__bool__`: conversion to bool, should return `True` or `False`; default behaviour: the object is considered truthy if len(object) != 0; If neither is defined all instances are considered truthy.
  - `__int__`, `__float__`, `__complex__`: conversion to the appropriate types if defined
  - `__index__`: e.g. when the object should be used for slicing
- **unary math**:
  - `__neg__` (-), `__pos__` (+), `__abs__` (abs), 
- **arithmetic**:
  - `__add__` (+), `__sub__` (-), `__mul__` (*), `__truediv__` (/), `__floordiv__` (//), `__mod__` (%), `__divmod__` (divmod), `__pow__` (**): 'normal' arithmetic
  - `__radd__` (+), `__rsub__` (-), `__rmul__` (*), `__rtruediv__` (/), `__rfloordiv__` (//), `__rmod__` (%), `__rdivmod__` (divmod), `__rpow__` (**): 'right/reversed' arithmetic, if normal is `NotImplemented`
  - `__iadd__` (+), `__isub__` (-), `__imul__` (*), `__itruediv__` (/), `__ifloordiv__` (//), `__imod__` (%), `__ipow__` (**): in-place modification
  - `__round__` (round), `__trunc__` (math.trunc), `__floor__` (math.floor), `__ceil__` (math.ceil): precision changes
- **bitwise operators**:
  -  `__invert__` (~): unary operators
  -  `__lshift__` (<<), `__rshift__` (>>), `__and__` (&), `__or__` (|), `__xor__` (^): normal binary operators
  -  `__rlshift__` (<<), `__rrshift__` (>>), `__rand__` (&), `__ror__` (|), `__rxor__` (^): 'right/reversed' binary operators, if normal is `NotImplemented`
  -  `__ilshift__` (<<), `__irshift__` (>>), `__iand__` (&), `__ior__` (|), `__ixor__` (^): in-place binary operators

- **Containers**:
  - `__len__`: return the number of elements in the container, used by builting `len`
  - `__getitem__`: used to implement `[]` with index/slice (sequence-like) or key (mapping-like)
  - `__setitem__`: element modification, otherwise as above
  - `__delitem__`: used to implement `del object[key]`, otherwise as above
  - `__iter__`, `__reversed__`: used to iterate over the elements in the container (eg by `for element in object`). Implement `reversed` only if your implementation is more efficient that backwards iteration.
  - `__contains__`: for tests like `element in container` if those can be implemented more efficiently than by iteration over all elements

- **emulating callables**:
  - `__call__`: makes instance objects callable as functions

- **context management**:
  - `__enter__`, `__exit__`: used to implement `with` blocks (discussed last week)
- **attribute access**:
  - `__getattribute__`: Called unconditionally to implement attribute accesses for instances of the class.
  - `__getattr__`:  Called when the default attribute access fails with an AttributeError
  - `__setattr__`: Called when an attribute assignment is attempted. Replaces the default behaviour (storing the value in the class dict)
  - `__delattr__`: Like `__setattr__` except for deletion (`del object.attribute`)
  - `__dir__`: Called when dir() is called on the object. A sequence must be returned. dir() converts the returned sequence to a list and sorts it.

### Subclassing built-in types can be tricky
- there are problems when subclassing built-in classes where method delegation is implemented in C for the CPython interpeter
- special classes exist which should be used instead
- This /is/ implementation-dependent, and looks differently if using eg PyPy.

In [ ]:
class DoubleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [ ]:
dd = DoubleDict(one=1)
dd['two'] = 2
dd.update(three=3)

In [ ]:
dd  # whut?!

In [ ]:
from collections import UserDict
class DoubleDict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [ ]:
dd = DoubleDict(one=1)
dd['two'] = 2
dd.update(three=3)

In [ ]:
dd

## Some general advice on classes and object oriented programming

### Interfaces
- An interface is a definition of a set of methods and functionality that must be supported by a class
- A class is said to 'implement' an interface if it supports all required methods/attributes
- In Python there are no explicit interfaces
- And you shouldn't worry too much about them anyways, just ducktyping is the Python way.
- If you do want to worry about interfaces look at Abstract Base Classes (abc) -- next week

### Mixins
- A mixin is a class that is 'designed to provide method implementations for reuse by multiple **unrelated** subclasses without implying an "is-a" relationship.
- That means it is **not** designed to be a generalization of it's subclasses
- Does not define a new 'object', just bundles methods for re-use.
- A mixin should never be instantiated directly
- Concrete Classes should never inherit **only** from a mixin
- Each mixin should provide a single specific behaviour
- There is no (syntatic) way to indentify mixins. Instead you should name any such class with a name ending on `Mixin`

# Inheritance
- Don't subclass from more than one concrete class. If your class is inheriting from one concrete class already, all other ancestors should be Mixins
- If you are writing complex class hierarchies (without being the developer of some big framework) you are probably over-engineering. Keep it simple.
- Favour composition over inheritance. It often makes more sense to hold a reference to an instance of some other class in one of your instance attributes.
  Only subclass if your object really **is** a specialization of the superclass.